# DS-SF-30 | Codealong 03: Databases, Scrapping, and APIs; Part B - NoSQL Databases and `CouchDB`

## Administration Web Interface

> ## `CouchDB`'s Administration Web Interface, called `Fauxton` is at http://localhost:5984/_utils

## A brief overview of `CouchDB`'s bare-bone API using the command-line utility `curl`

> ### Hello World!

In [ ]:
!curl -X GET http://localhost:5984

> ### Get the list of databases

In [ ]:
!curl -X GET http://localhost:5984/_all_dbs

> ## Create a new database called `zillow`

In [ ]:
!curl -X PUT http://localhost:5984/zillow

> ## Check out the new database

In [ ]:
!curl -X GET http://localhost:5984/zillow

It's a new database so it is empty (`"doc_count":0`).  Let's now populate it.

> ## Bulk import

In [ ]:
!curl -d "@../datasets/dataset-03-zillow.json" -H "Content-Type: application/json" -X POST http://localhost:5984/zillow/_bulk_docs

Let's check it again...

In [ ]:
!curl -X GET http://localhost:5984/zillow

We can see that we now have 1,000 documents in it (`"doc_count":1000`).  Let's have a look at a couple of documents.

> ## `_id`

In [ ]:
!curl -X GET http://localhost:5984/zillow/15065032

`15065032` refers to the Zillow identifier of our dataset but how did `CouchDB` know how to use it?  For the bulk import, we set the identifier of each document with `_id`.

## List of all documents within a database

In [ ]:
!curl -X GET http://localhost:5984/zillow/_all_docs

> Now with the document contents

In [ ]:
!curl -X GET http://localhost:5984/zillow/_all_docs?include_docs=true

## Let's now using `CouchDB`'s `MapReduce` to compute the average sale price of these properties based on their number of bedrooms

> Using `Fauxton`, let's create a new view: http://localhost:5984/_utils/#/database/zillow/new_view

Copy/paste the following (JavaScript) code for the `Map function`.

```javascript
function(property) {
    if (property.beds) {
        beds = property.beds;
    } else if (property.is_a_studio == true) {
        beds = 0;
    } else {
        return;
    }
    if (property.sale.price.unit == '$') {
        emit(beds, [property.sale.price.value / Math.pow(10, 6), 1]);
        return;
    }
    if (property.sale.price.unit == '$M') {
        emit(beds, [property.sale.price.value, 1]);
        return;
    }
}
```

Copy/paste the following (JavaScript) code for the `Reduce (optional)` function (choose `CUSTOM`).

```javascript
function(keys, values) {
    weighted_price = 0
    weight = 0
        values.forEach(function (value) {
            weighted_price += value[0] * value[1];
            weight += value[1];
        });
     return [weighted_price / weight, weight];
}
```

Click on `Options` (upper right), check `Reduce` and `Run Query`.  Do these numbers match what we got from the `SQL` section?

## Delete a database

In [ ]:
!curl -X DELETE http://localhost:5984/zillow